## Imports 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import tree, svm
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

## Data

In [2]:
bottle1 = pd.read_csv("bottle_data_2010-14.csv")
bottle2 = pd.read_csv("bottle_Data_2015to19.csv", header=None)
cast = pd.read_csv("cast_Data_2015to19.csv")

/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,48,49,50,51,52,53,54,55,56,57,58,59,60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
bottle = pd.read_csv("bottle.csv", encoding='utf-8')

/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (47,61) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(bottle1.columns)
bottle1.head()
bottle1.info()

Index(['Cst_Cnt', 'Btl_Cnt', 'Sta_ID', 'Depth_ID', 'Depthm', 'T_degC',
       'Salnty', 'O2ml_L', 'STheta', 'O2Sat', 'Oxy_µmol/Kg', 'BtlNum',
       'RecInd', 'T_prec', 'T_qual', 'S_prec', 'S_qual', 'P_qual', 'O_qual',
       'SThtaq', 'O2Satq', 'ChlorA', 'Chlqua', 'Phaeop', 'Phaqua', 'PO4uM',
       'PO4q', 'SiO3uM', 'SiO3qu', 'NO2uM', 'NO2q', 'NO3uM', 'NO3q', 'NH3uM',
       'NH3q', 'C14As1', 'C14A1p', 'C14A1q', 'C14As2', 'C14A2p', 'C14A2q',
       'DarkAs', 'DarkAp', 'DarkAq', 'MeanAs', 'MeanAp', 'MeanAq', 'IncTim',
       'LightP', 'R_Depth', 'R_TEMP', 'R_Sal', 'R_DYNHT', 'R_Nuts',
       'R_Oxy_µmol/Kg', 'DIC1', 'DIC2', 'TA1', 'TA2', 'pH1', 'pH2',
       'DIC Quality Comment'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38819 entries, 0 to 38818
Data columns (total 62 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Cst_Cnt              38819 non-null  int64  
 1   Btl_Cnt              3

In [5]:
#extracting features we will use
bottle1['nitrogen'] = bottle1['NO3uM'] + bottle1['NO2uM'] + bottle1['NH3uM']
temp = bottle1[['Depthm', 'T_degC', 'SiO3uM', 'PO4uM', 'nitrogen', 'ChlorA']]
temp.dropna(inplace=True)
X = temp[['Depthm', 'T_degC', 'SiO3uM', 'PO4uM', 'nitrogen']]
y = temp['ChlorA']


X.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 24512 entries, 0 to 38818
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Depthm    24512 non-null  int64  
 1   T_degC    24512 non-null  float64
 2   SiO3uM    24512 non-null  float64
 3   PO4uM     24512 non-null  float64
 4   nitrogen  24512 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 1.1 MB


/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [6]:
display(X) #inputs
display(y) #targets

,Depthm,T_degC,SiO3uM,PO4uM,nitrogen
0,0,16.360,2.80,0.32,0.000
1,2,16.360,2.80,0.32,0.000
2,5,15.920,3.00,0.34,0.000
3,10,15.620,3.30,0.37,0.120
4,11,15.600,3.30,0.37,0.120
...,...,...,...,...,...
38814,0,19.672,2.96,0.33,0.182
38815,2,19.672,2.96,0.33,0.142
38816,5,19.022,2.98,0.33,0.290
38817,10,18.722,3.06,0.34,0.199


0        0.770
1        0.770
2        0.950
3        1.320
4        1.380
         ...  
38814    0.611
38815    0.611
38816    0.356
38817    0.486
38818    0.735
Name: ChlorA, Length: 24512, dtype: float64

In [7]:
y.isna().sum()

0

## Trying out different algorithms from sci-kit learn

In [10]:
# goal: predict chlorA amount using inputs, and check performance by target - unscaled 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

reg = LinearRegression().fit(X_train, y_train)
print('Linear regression:')
print(reg.score(X_train, y_train))
print(reg.score(X_test, y_test))

tree = tree.DecisionTreeRegressor().fit(X_train,y_train)
print('Decision tree:')
print(tree.score(X_train, y_train))
print(tree.score(X_test, y_test))


svm = svm.SVR().fit(X_train,y_train)
print('SVM:')
print(svm.score(X_train,y_train))
print(svm.score(X_test,y_test))

Linear regression:
0.15614814911926944
0.1568742981037694


AttributeError: 'DecisionTreeRegressor' object has no attribute 'DecisionTreeRegressor'

In [ ]:
nn = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
y_pred = nn.predict(X_test)
print('NN:')
print(nn.score(X_train,y_train))
print(nn.score(X_test, y_test))
print(r2_score(y_test,y_pred))


In [9]:
# scaled
StandardScaler().fit_transform(X_train)
StandardScaler().fit_transform(X_test)

reg = LinearRegression().fit(X_train, y_train)
print('Linear regression:')
print(reg.score(X_train, y_train))
print(reg.score(X_test, y_test))

tree = tree.DecisionTreeRegressor().fit(X_train,y_train)
print('Decision tree:')
print(tree.score(X_train, y_train))
print(tree.score(X_test, y_test))


svm = svm.SVR().fit(X_train,y_train)
print('SVM:')
print(svm.score(X_train,y_train))
print(svm.score(X_test,y_test))

nn = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
y_pred = nn.predict(X_test) # this is for r2_score
print('NN:')
print(nn.score(X_train,y_train))
print(nn.score(X_test, y_test))
print(r2_score(y_test,y_pred)) #same as above line


Linear regression:
0.15614814911926944
0.1568742981037694
Decision tree:
0.999999953175804
0.34315833553488184
SVM:
0.1468039577008161
0.135878013934841
NN:
0.37286049108764974
0.34482422554367453
0.34482422554367453
